In [1]:
import torch
import numpy as np
import cv2
import os
import random as rand
import torchvision
import pandas as pd
from tqdm import tqdm
from torch import nn, Tensor
import matplotlib.pyplot as plt
from typing import Optional
from torch.nn import functional as F
from torchvision.transforms import v2 as T
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from math import ceil

In [2]:
class CIFAR(Dataset):
    def __init__(self, path="/scratch/s25090/archive/cifar-10/train", dataset:Optional[list]=None):
        super().__init__()
        self.path = path
        self.files = os.listdir(self.path) if dataset is None else dataset
        self.T = T.Compose([
           T.ToImage(), 
           T.ToDtype(torch.float32, scale=True),
           T.Resize((32, 32)),
           T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def __len__(self,):
        return len(self.files)
    
    def __getitem__(self, idx):
        file = self.files[idx]
        img_path = os.path.join(self.path, file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.T(img)
        return img

In [3]:
class GenBlock(nn.Module):
    def __init__(self, in_channel, out_channel, is_final):
        super().__init__()
        layers = [
            nn.Conv2d(in_channel, (out_channel+in_channel)//2, 3, 1, 1),
            nn.BatchNorm2d((out_channel+in_channel)//2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d((in_channel+out_channel)//2, out_channel, 3, 1, 1)
        ]
        
        if not is_final:
            layers.append(nn.BatchNorm2d(out_channel))
            layers.append(nn.LeakyReLU(0.2, inplace=True))

        layers.append(nn.UpsamplingNearest2d(scale_factor=2))
        self.layer = nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)

class DisBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channel, (out_channel+in_channel)//2, 3, 1, 1),
            nn.BatchNorm2d((out_channel+in_channel)//2),
            nn.LeakyReLU(0.2),
            nn.Conv2d((out_channel+in_channel)//2, out_channel, 3, 1, 1),
            nn.BatchNorm2d(out_channel),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2)
        )
    
    def forward(self, x):
        return self.layer(x)

class ResGenBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.conv_block = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        )
        
        self.shortcut = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False)
        )

    def forward(self, x):
        return self.conv_block(x) + self.shortcut(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super().__init__()
        self.initial_linear = nn.Linear(z_dim, 1024 * 4 * 4)
        
        self.net = nn.Sequential(
            GenBlock(1024, 512, is_final=False), 
            GenBlock(512, 256, is_final=False),  
            GenBlock(256, 128,  is_final=False), 
            GenBlock(128,  64,  is_final=False),
            GenBlock(64,  64,  is_final=True),
        )
        
        self.final_layer = nn.Sequential(
            nn.Conv2d(64, 3, kernel_size=3, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        if len(z.shape) > 2:
            z = z.view(z.size(0), -1)
            
        x = self.initial_linear(z)
        x = x.view(-1, 1024, 4, 4)
        x = self.net(x)
        return self.final_layer(x)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            DisBlock(3, 32),   
            DisBlock(32, 64),
            DisBlock(64, 128),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 1) 
        )

    def forward(self, x):
        x = self.net(x)
        return self.classifier(x)

class ResNetGenerator(nn.Module):
    def __init__(self, z_dim=100, base_channels=256):
        super().__init__()
        self.linear = nn.Linear(z_dim, 4 * 4 * base_channels)
        self.base_channels = base_channels

        self.blocks = nn.Sequential(
            ResGenBlock(base_channels, base_channels),    
            ResGenBlock(base_channels, base_channels // 2), 
            ResGenBlock(base_channels // 2, base_channels // 4),
        )
        
        self.final_layer = nn.Sequential(
            nn.BatchNorm2d(base_channels // 4),
            nn.ReLU(inplace=True),
            nn.Conv2d(base_channels // 4, 3, 3, 1, 1), 
            nn.Tanh() 
        )

    def forward(self, z):
        if z.ndim > 2: z = z.view(z.size(0), -1)
            
        x = self.linear(z)
        x = x.view(-1, self.base_channels, 4, 4)
        x = self.blocks(x)
        return self.final_layer(x)

class WGANModel(nn.Module):
    def __init__(self, z_dim=100, is_res=True):
        super().__init__()
        self.generator = Generator(z_dim) if not is_res else ResNetGenerator()
        self.discriminator = Discriminator()
        self.z_dim = z_dim

    def forward(self, z):
        return self.generator(z)

    def compute_discriminator_loss(self, real_imgs, z):
        with torch.no_grad():
            fake_imgs = self.generator(z).detach()

        real_logits = self.discriminator(real_imgs)
        fake_logits = self.discriminator(fake_imgs)

        d_loss = -(torch.mean(real_logits) - torch.mean(fake_logits))
        
        return d_loss

    def compute_generator_loss(self, z):
        fake_imgs = self.generator(z)
        
        fake_logits = self.discriminator(fake_imgs)
        
        g_loss = -torch.mean(fake_logits)
        
        return g_loss, fake_imgs

In [ ]:
LEARNING_RATE = 5e-5        
WEIGHT_CLIP = 0.01
N_CRITIC = 5  
DEVICE = 'cuda:3' if torch.cuda.is_available() else 'cpu'
epochs = 300

gan_model = WGANModel()
gan_model = gan_model.to(DEVICE)

opt_gen = torch.optim.RMSprop(gan_model.generator.parameters(), lr=LEARNING_RATE)
opt_dis = torch.optim.RMSprop(gan_model.discriminator.parameters(), lr=LEARNING_RATE)

train_dataset = CIFAR()
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

gen_loss_list = []
dis_loss_list = []

for epoch in range(epochs):
    gan_model.train()
    tqdm_data = tqdm(train_loader, desc=f"Epoch-{epoch+1}/{epochs}")
    
    batch_gen_loss = 0
    batch_dis_loss = 0
    
    for batch_idx, (real_img) in enumerate(tqdm_data):
        real_img = real_img.to(DEVICE)
        bs = real_img.size(0)

        for param in gan_model.discriminator.parameters():
            param.requires_grad = True

        z_dis = torch.randn(bs, 100).to(DEVICE)
        opt_dis.zero_grad()
        
        dis_loss = gan_model.compute_discriminator_loss(real_img, z_dis)
        dis_loss.backward()
        opt_dis.step()

        for p in gan_model.discriminator.parameters():
            p.data.clamp_(-WEIGHT_CLIP, WEIGHT_CLIP)
        
        batch_dis_loss += dis_loss.item()

        if batch_idx % N_CRITIC == 0:
            for param in gan_model.discriminator.parameters():
                param.requires_grad = False # Freeze D to save computation
            
            z = torch.randn(bs, 100).to(DEVICE)
            opt_gen.zero_grad()
            
            gen_loss, fake_img = gan_model.compute_generator_loss(z)
            gen_loss.backward()
            opt_gen.step()
            
            batch_gen_loss += gen_loss.item()
            current_gen_loss = gen_loss.item() # For tqdm
        else:
            current_gen_loss = batch_gen_loss / (batch_idx + 1) if batch_idx > 0 else 0

        tqdm_data.set_postfix({
            "GenLoss": current_gen_loss,
            "DisLoss": dis_loss.item()
        })

    avg_gen_loss = batch_gen_loss / (len(train_loader) / N_CRITIC)
    avg_dis_loss = batch_dis_loss / len(train_loader)
    
    gen_loss_list.append(avg_gen_loss)
    dis_loss_list.append(avg_dis_loss)

    print(f"Generator Loss: {avg_gen_loss:.4f}\nDiscriminator Loss: {avg_dis_loss:.4f}")

    if (epoch + 1) % 10 == 0:
        torch.save(gan_model.state_dict(), f'/scratch/s25090/wgan_outputs/weights/Experiment1/gan_epoch_{epoch+1}.pth')
        gan_model.eval()
        with torch.no_grad():
            test_z = torch.randn(8, 100).to(DEVICE)
            gan_image = gan_model.generator(test_z)
            
            comparison = torch.cat([real_img[:8], gan_image[:8]], dim=0)
            grid = make_grid(comparison.cpu(), nrow=8, padding=2, normalize=True)
            
            plt.figure(figsize=(12, 4))
            plt.imshow(grid.permute(1, 2, 0))
            plt.axis('off')
            plt.title(f'Top: Original | Bottom: Generated Image (Epoch {epoch+1})')
            plt.savefig(f"/scratch/s25090/wgan_outputs/plots/Experiment1/Epoch-{epoch+1}.png")
            plt.close()

plt.figure(figsize=(10, 5))
plt.title("Generator vs Discriminator Loss (WGAN)")
plt.plot(gen_loss_list, label="Generator")
plt.plot(dis_loss_list, label="Discriminator")
plt.xlabel("Epochs")
plt.ylabel("Wasserstein Loss")
plt.savefig(f"/scratch/s25090/wgan_outputs/plots/Experiment1_loss.png")
plt.legend()
plt.show()

Epoch-1/300: 100%|██████████| 391/391 [02:41<00:00,  2.43it/s, GenLoss=0.0106, DisLoss=-8.13e-6] 


Generator Loss: 0.0107
Discriminator Loss: 0.0005


Epoch-2/300: 100%|██████████| 391/391 [01:17<00:00,  5.06it/s, GenLoss=0.0112, DisLoss=-8.43e-5] 


Generator Loss: 0.0109
Discriminator Loss: -0.0000


Epoch-3/300: 100%|██████████| 391/391 [00:49<00:00,  7.92it/s, GenLoss=0.114, DisLoss=-0.0601]    


...

Epoch-52/300: 100%|██████████| 391/391 [00:33<00:00, 11.84it/s, GenLoss=-0.00583, DisLoss=-0.0774]


Generator Loss: 0.0358
Discriminator Loss: -0.0650


Epoch-53/300: 100%|██████████| 391/391 [00:31<00:00, 12.54it/s, GenLoss=-0.0359, DisLoss=-0.0729] 


Generator Loss: 0.0369
Discriminator Loss: -0.0640


Epoch-54/300:  29%|██▉       | 115/391 [00:46<26:40,  5.80s/it, GenLoss=0.00851, DisLoss=-0.063] 